<a href="https://colab.research.google.com/github/soedinglab/spacedust/blob/master/Spacedust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spacedust: *de novo* discovery of conserved gene clusters in microbial genomes
---

<img src="https://raw.githubusercontent.com/soedinglab/spacedust/master/.github/spacedust.png" height="200" align="right" style="height:200px"/>

Spacedust is a modular toolkit for identification of conserved gene clusters among multiple genomes based on homology and conservation of gene neighborhood. Spacedust adapts the fast and sensitive structure comparisons of [Foldseek](https://github.com/steineggerlab/foldseek) and homology search capabilities of [MMseqs2](https://github.com/soedinglab/MMseqs2). It introduces a novel approach of aggregating sets of homologous hits between pairs of genomes and identifies cluster of hits with conserved gene neighborhood between each using agglomerative hierarchical clustering algorithm. Spacedust is GPLv3-licensed open source software implemented in C++ and available for Linux and macOS. The software is designed to run efficiently on multiple cores.

<!-- [Zhang, R., Mirdita, M., Levy Karin, E., Norroy, C., Galiez, C., & Söding, J.  SpacePHARER: Sensitive identification of phages from CRISPR spacers in prokaryotic hosts. Bioinformatics, doi:10.1093/bioinformatics/btab222 (2021).](https://doi.org/10.1093/bioinformatics/btab222) -->

**Input**

* Each file should contain one genome. Genomes should be uploaded as:
  * `.fna` files and turn on the `run_prodigal` option, or
  * `.faa` files with header in Prodigal format
  

In [9]:
#@title Set parameters, hit `Runtime` -> `Run all`, wait for file input box to appear below this cell to upload  1) all your query genome files and 2) your target genome files
jobname = 'test' #@param {type:"string"}
#@markdown ---
#@markdown Input options (`target_db` is ignored if `input_mode` is `all-against-all`)
input_mode = "query-target" #@param ["query-target", "all-against-all"]
target_db="KEGG_70" #@param ["self-uploaded", "KEGG_70"]
search_mode = "MMseqs2" #@param ["MMseqs2"]
run_prodigal = True #@param {type:"boolean"}
#@markdown ---
#@markdown Advanced options
max_gene_gap = 3 #@param {type:"integer"}
num_iterations = 1 #@param {type:"integer"}
#@markdown ---
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form. A input box will appear below this text. Upload your genomes.

jobname = "".join(jobname.split())

input_type = 0
if input_mode == "all-against-all":
  input_type = 1

target_type = 0
if target_db != "self-uploaded":
  target_type = 1

search_type = 0
if search_mode == "Foldseek":
  search_type = 1


import os
import shutil
from google.colab import files

# Clear and recreate the input directory
shutil.rmtree(jobname + "_input", ignore_errors=True)
os.makedirs(jobname + "_input", exist_ok=True)

# Upload files and count them
uploaded = files.upload()
num_queries = len(uploaded.keys())

# Check if the number of queries is more than 1 for all-against-all mode
if input_type == 1 and num_queries <= 1:
    raise ValueError("Error: 'all-against-all' mode requires more than 1 query file.")

# Move uploaded files to the input directory
for fn in uploaded.keys():
    os.replace(fn, os.path.join(jobname + "_input", fn))

# Handle target database upload if needed
if input_type == 0 and target_type == 0:
    shutil.rmtree(jobname + "_target", ignore_errors=True)
    os.makedirs(jobname + "_target", exist_ok=True)
    uploaded = files.upload()
    for fn in uploaded.keys():
        os.replace(fn, os.path.join(jobname + "_target", fn))


Saving sequence.fna to sequence (1).fna


In [10]:
#@title Download dependencies and databases
%%bash -s $jobname $input_type $search_type $run_prodigal $target_type $target_db
JOBNAME=$1
INPUT_TYPE=$2
SEARCH_TYPE=$3
NEED_PRODIGAL=$4
TARGET_TYPE=$5
TARGET_DB=$6

if [ ! -e SPACEDUST_READY ]; then
  wget -q wget https://mmseqs.com/spacedust/spacedust-linux-avx2.tar.gz
  tar -xzf spacedust-linux-avx2.tar.gz
  rm -f spacedust-linux-avx2.tar.gz
  touch SPACEDUST_READY
fi

if [ ! -e FOLDSEEK_READY ] && [ "${SEARCH_TYPE}" = "1" ]; then
  wget -q wget https://mmseqs.com/foldseek/foldseek-linux-avx2.tar.gz
  tar -xzf foldseek-linux-avx2.tar.gz
  rm -f foldseek-linux-avx2.tar.gz
  mv foldseek/bin/foldseek spacedust/bin
  rm -rf foldseek
  touch FOLDSEEK_READY
fi

if [ ! -e PRORIGAL_READY ] && [ "${NEED_PRODIGAL}" = "True" ]; then
  wget -q wget https://github.com/hyattpd/Prodigal/releases/download/v2.6.3/prodigal.linux
  cp prodigal.linux /usr/local/bin/prodigal
  chmod +x /usr/local/bin/prodigal
  touch PRODIGAL_READY
fi

mkdir -p database
cd database
if [ ! -e SPACEDUST_DB_READY ] && [ "${INPUT_TYPE}" = "0" ] && [ "${TARGET_TYPE}" = "1" ]; then
  wget -N -np -nv http://wwwuser.gwdg.de/~compbiol/spacedust/${TARGET_DB}.tar.gz
  tar -xzf ${TARGET_DB}.tar.gz
  rm -f ${TARGET_DB}.tar.gz
  touch ../SPACEDUST_DB_READY
fi

2024-10-04 16:17:13 URL:https://wwwuser.gwdguser.de/~compbiol/spacedust/KEGG_70.tar.gz [972659708/972659708] -> "KEGG_70.tar.gz" [1]


In [ ]:
#@title Run Spacedust
%%bash -s $jobname $input_type $search_type $run_prodigal $target_type $max_gene_gap
JOBNAME=$1
INPUT_TYPE=$2
SEARCH_TYPE=$3
NEED_PRODIGAL=$4
TARGET_TYPE=$5
MAX_GENE_GAP=$6
NUM_ITERATIONS=$7



if [ ! -e PRORIGAL_FAA_READY ] && [ "${NEED_PRODIGAL}" = "True" ] ; then
    if ! ls "${JOBNAME}_input/"*.fna 1> /dev/null 2>&1; then
      echo "Error: No query .fna file is found! Please make sure you input the query files with the correct extension!"
      exit 1
    fi

    for filename in "${JOBNAME}_input/"*.fna; do
      base=$(basename "$filename" .fna)
      if [ ! -f "${JOBNAME}_input/$base.faa" ]; then
          prodigal -i "$filename" -a "${JOBNAME}_input/$base.faa" &> /dev/null
      fi
    done

    if [ "$INPUT_TYPE" = "0" ] && [ "$TARGET_TYPE" = "0" ] ; then

    if ! ls "${JOBNAME}_input/"*.fna 1> /dev/null 2>&1; then
      echo "Error: No target .fna file is found! Please make sure you input the target files with the correct extension!"
      exit 1
    fi

      for filename in "${JOBNAME}_target/"*.fna; do
        base=$(basename "$filename" .fna)
        if [ ! -f "${JOBNAME}_target/$base.faa" ]; then
            prodigal -i "$filename" -a "${JOBNAME}_target/$base.faa" &> /dev/null
        fi
      done
    fi
    touch PRODIGAL_FAA_READY
fi

if [ "$INPUT_TYPE" = "1" ]; then
    spacedust/bin/spacedust createsetdb "${JOBNAME}_input/"*.faa database/${JOBNAME}_input tmp -v 0 &> /dev/null
    spacedust/bin/spacedust clustersearch database/${JOBNAME}_input database/${JOBNAME}_input "$JOBNAME" tmp --filter-self-match --search-mode "$SEARCH_TYPE" --max-gene-gap "$MAX_GENE_GAP" -v 0 &> /dev/null
else
    spacedust/bin/spacedust createsetdb "${JOBNAME}_input/"*.faa database/${JOBNAME}_input tmp -v 0 &> /dev/null
    if [ "$TARGET_TYPE" = "0" ]; then
      spacedust/bin/spacedust createsetdb "${JOBNAME}_target/"*.faa database/${JOBNAME}_db tmp -v 0 &> /dev/null
      spacedust/bin/spacedust clustersearch database/${JOBNAME}_input database/${JOBNAME}_db "$JOBNAME" tmp --search-mode "$SEARCH_TYPE" --max-gene-gap "$MAX_GENE_GAP"  -v 0 &> /dev/null
    else
      spacedust/bin/spacedust clustersearch database/${JOBNAME}_input database/keggclusterdb "$JOBNAME" tmp --search-mode "$SEARCH_TYPE"  --max-gene-gap "$MAX_GENE_GAP" -v 0
    fi
fi

spacedust/bin/spacedust prefixid tmp/latest/clusters "${JOBNAME}_pref" --tsv -v 0 &> /dev/null

awk '{ print $2"\t"$3 }' "${JOBNAME}_pref" > qid_tid
awk '{ print $1 }' "${JOBNAME}_pref" > cluid
rm "${JOBNAME}_pref"

awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$2; next} {$1=clr[$1]; print}' database/${JOBNAME}_input".lookup" qid_tid > 1
if [ "$INPUT_TYPE" = "1" ]; then
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$2; next} {$2=clr[$2]; print}' database/${JOBNAME}_input".lookup" 1 > qname_tname
elif [ "$TARGET_TYPE" = "0" ]; then
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$2; next} {$2=clr[$2]; print}' database/${JOBNAME}_db".lookup" 1 > qname_tname
else
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$2; next} {$2=clr[$2]; print}' database/keggclusterdb.lookup 1 > qname_tname
fi
sed -i 's/NZ_/NZ./g' qname_tname
sed -i 's/NC_/NC./g' qname_tname
tr '_' '\t' <qname_tname > qname_tname_sep
rm 1
rm qname_tname

awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$3; next} {$1=clr[$1]; print}' database/${JOBNAME}_input".lookup" qid_tid > 2
if [ "$INPUT_TYPE" = "1" ]; then
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$3; next} {$2=clr[$2]; print}' database/${JOBNAME}_input".lookup" 2 > qset_tset
elif [ "$TARGET_TYPE" = "0" ]; then
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$3; next} {$2=clr[$2]; print}' database/${JOBNAME}_db".lookup" 2 > qset_tset
else
    awk 'BEGIN{OFS=FS="\t"} NR==FNR{clr[$1]=$3; next} {$2=clr[$2]; print}' database/keggclusterdb.lookup 2 > qset_tset
fi
rm 2

paste cluid qset_tset qid_tid qname_tname_sep > "${JOBNAME}_plot"
rm qid_tid
rm qname_tname_sep
rm qset_tset
rm cluid

spacedust/bin/spacedust prefixid "database/${JOBNAME}_input" "database/${JOBNAME}_input_pref" --tsv -v 0 &> /dev/null

In [ ]:
#@title Show cluster matches
from pathlib import Path
from IPython.display import HTML
from IPython.core.display import display
from pathlib import Path

html = '''
<style>
tbody tr.head {
  font-weight: bold;
  background-color: #f2f2f2;
}
</style>
<table style='text-align:left'>
  <thead>
    <tr>
      <th>Cluster Match ID</th>
      <th>Query Acc.</th>
      <th>Target Acc.</th>
      <th>Cluster Match Pval</th>
      <th colspan="8">Num. Hits</th>
    </tr>
    <tr>
      <th colspan='2'>Query ID</th>
      <th>Target ID</th>
      <th>SeqId</th>
      <th>Eval</th>
      <th>qStart</th>
      <th>qEnd</th>
      <th>qLength</th>
      <th>tStart</th>
      <th>tEnd</th>
      <th>tLength</th>
      <th>Aln Cigar</th>
    </tr>
  </thead>
<tbody>
'''

for line in Path(jobname).read_text().split('\n'):
  if len(line) == 0:
    continue
  if line[0] == '#':
    cols = line.split('\t')
    if len(cols) == 6:
      html += "<tr class='head'>"
      html += "<td>" + cols[0][1:] + "</td>"
      html += "<td>" + cols[1] + "</td>"
      html += "<td>" + cols[2] + "</td>"
      html += "<td>" + cols[4] + "</td>"
      html += "<td colspan='8'>" + cols[5] + "</td>"
      html += "</tr>\n"
  elif line[0] == '>':
    cols = line.split('\t')
    if len(cols) == 12:
      html += "<tr>"
      html += "<td colspan='2'>" + cols[0][1:] + "</td>"
      html += "<td>" + cols[1] + "</td>"
      html += "<td>" + cols[3] + "</td>"
      html += "<td>" + cols[4] + "</td>"
      html += "<td>" + cols[5] + "</td>"
      html += "<td>" + cols[6] + "</td>"
      html += "<td>" + cols[7] + "</td>"
      html += "<td>" + cols[8] + "</td>"
      html += "<td>" + cols[9] + "</td>"
      html += "<td>" + cols[10] + "</td>"
      html += "<td>" + cols[11] + "</td>"
    html += "</tr>\n"
html += '''
</tbody>
</table>
'''
display(HTML(html))

In [ ]:
#@title Download result

download_path = "/content/"+ jobname
files.download(download_path)

# **Visualization**

In [ ]:
#@title Prepare result for visualization
!python3 -m pip install -q -U ipympl
!python3 -m pip install -q -U rpy2==3.5.1
import os
import shutil
from google.colab import files
import numpy as np
import pandas as pd
import seaborn as sns

matchhit = pd.read_csv(jobname + "_plot", sep="\t", names=['cluid','qsetid', 'tsetid', 'qseqid','tseqid','qname', 'qid_p','qid','qstart','qend', 'tname','tid','tstart','tend'])

lookup = pd.read_csv("database/" + jobname + "_input.lookup", sep="\t", names=['seqid','header','setid'])

all_seq = pd.read_csv("database/" + jobname + "_input_pref", sep="\t", names=['id','seq'], header=None, dtype={'id' : int, 'seq': str})


In [ ]:
#@title Select query genome

from ipywidgets import interact, widgets

# Load your data file into a DataFrame
# Replace 'your_file.csv' with the actual file path
df = pd.read_csv('database/'+jobname + '_input.source', sep="\t",names=['query_genome_id','query_genome_name'],dtype={'query_genome_id' : int, 'query_genome_name': str})

values = df['query_genome_name']

if len(values) == 1 :
    query_genome = 0
else:
  # Initialize a global variable to store the selected query_genome_id
  query_genome = None

  # Create an interactive dropdown menu
  @interact(selected_value=widgets.Dropdown(options=unique_values, description='Select Value:'))
  def set_query_genome_id(selected_value):
      global query_genome

      # Set query_genome_id based on the selected value
      query_genome = df.loc[df['query_genome_name'] == selected_value, 'query_genome_id'].iloc[0]

      # Display information based on the selected value and set query_genome_id
      selected_data = df[df['query_genome_name'] == selected_value]



#calculate match barplot
#create array for counting
matchhit_temp = matchhit[matchhit['qsetid'] == query_genome]
qid = matchhit_temp.drop_duplicates(['qid','tsetid'],keep= 'last')['qid'].to_numpy()
matchhit_array = np.zeros(qid.max()+1, dtype=int)
for i in qid:
    matchhit_array[i]+=1
#create array for counting neighboring pairs crossing clusters (conservation matrix)
qid = matchhit.sort_values(['cluid', 'qid'])['qid'].to_numpy()
tid = matchhit.sort_values(['cluid', 'qid'])['tid'].to_numpy()
cluid = matchhit.sort_values(['cluid', 'qid'])['cluid'].to_numpy()
matchpair_array = np.zeros(qid.max(), dtype=int)
for i in np.arange(len(qid)-1):
    if(cluid[i] == cluid[i+1]):
        if(qid[i] == qid[i+1]-1):
            matchpair_array[qid[i]]+=1
        else:
            if(abs(qid[i+1]-qid[i])==abs(tid[i+1]-tid[i])):
                for x in np.arange(qid[i],qid[i+1]):
                    matchpair_array[x]+=1

count=np.zeros(matchhit_array.max()+1)
for i in matchhit_array.tolist():
    count[i]+=1

#process lookup
lookup_temp= lookup[lookup['setid'] == query_genome]

lookup_temp['idx'] = lookup_temp['header'].str.split('_').str[-3].astype(int)
lookup_temp['qstart'] = lookup_temp['header'].str.split('_').str[-2].astype(int)
lookup_temp['qend'] = lookup_temp['header'].str.split('_').str[-1].astype(int)

In [ ]:
#@title Cluster matches heatmap/bar plot

Zoom = False #@param {type:"boolean"}
#@markdown Zoom in to the plot by setting a lower and upper bound of the protein id (ignored if `zoom` is not selected)
lower_bound = 1 #@param {type:"integer"}
upper_bound = 100 #@param {type:"integer"}

# Ensure that lower_bound is always smaller than upper_bound
if lower_bound >= upper_bound:
    raise ValueError("Lower bound must be smaller than upper bound")


#combined bar plot
%matplotlib widget

from google.colab import output
output.enable_custom_widget_manager()
import matplotlib.pyplot as plt
import math
import matplotlib.cm as cm
from matplotlib.widgets import Slider
import matplotlib.ticker as ticker

# from matplotlib.colors import ListedColormap
from ipywidgets import *

from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get the maximum protein ID and genome ID
max_protein_id = np.max(matchhit['qid'])
max_genome_id = np.max(matchhit['tsetid'])

# Create an empty matrix of the correct size
matrix = np.zeros((max_genome_id + 1, max_protein_id + 1))
matrix[query_genome,:] = 1
# Fill the matrix with the protein hits data
for _, row in matchhit.iterrows():
    matrix[row['tsetid'], row['qid']] = 1

# Create an empty matrix for the strand plot
strand_plot = np.zeros( len(lookup_temp['idx']), dtype=int)

# Loop through the gene dataframe to mark positive and negative strands
for _, row in lookup_temp.iterrows():
    if (row['qstart'] < row['qend']):
        # Set positive strand positions to 1
        strand_plot[row['idx']] = 1

# Create the figure and axes with adjusted height ratio
fig, (ax1, ax3, ax2) = plt.subplots(nrows=3, sharex=True, figsize=(10, 12), gridspec_kw={'height_ratios': [4, 0.1, 1]})

# Create the heatmap
heatmap = ax1.imshow(matrix, cmap='Blues', interpolation='none', aspect='auto')

# Set the title, y-axis label, and y-axis tick labels
# ax1.set_title('Presence/Absence Heatmap')
ax1.set_ylabel('Genome ID')
y_labels = range(0, max_genome_id + 1)
ax1.set_yticks(range(0, max_genome_id + 1))
ax1.set_yticklabels(y_labels)

# Use AutoLocator for x-axis ticks to adjust dynamically based on zoom level
ax1.xaxis.set_major_locator(ticker.AutoLocator())

# Use AutoLocator for y-axis ticks to adjust dynamically based on zoom level
ax1.yaxis.set_major_locator(ticker.AutoLocator())

# Set the x-axis tick labels and rotation
x_labels = range(0, max_protein_id + 1)
ax2.set_xticks(range(0, max_protein_id + 1))
ax2.set_xticklabels(x_labels, rotation=90)

# Add a bar plot below the heatmap
ax2.bar(np.arange(len(matchpair_array)), matchpair_array,width=1,align='edge',ecolor='black',color='lightpink')
ax2.bar(np.arange(len(matchhit_array)), matchhit_array,width=0.5,align='center')
ax2.set_ylabel('Hits Count')
ax2.set_xlabel('Query Protein Position Index')

# Adjust the limits of the x-axis to match the heatmap
if Zoom:
    ax2.set_xlim(lower_bound-0.5, upper_bound + 0.5)
else:
    ax2.set_xlim(-0.5, max_protein_id+1 - 0.5)

ax2.set_yscale('log')

# Use AutoLocator for x-axis ticks to adjust dynamically based on zoom level
ax2.xaxis.set_major_locator(ticker.AutoLocator())

# Add the strand plot as a thin line
cmap_binary = cm.binary
ax3.imshow(strand_plot.reshape(1, -1), cmap=cmap_binary, aspect='auto')
ax3.yaxis.set_ticks([])
ax3.set_ylabel('Strand')

# Enable interactive mode
plt.ion()

# Function to resize the plot when zooming in
def on_zoom(event):
    current_xlim = ax1.get_xlim()
    current_ylim = ax1.get_ylim()
    current_ylim2 = ax2.get_ylim()
    ax1.set_xlim(*current_xlim)
    ax1.set_ylim(*current_ylim)
    ax2.set_ylim(*current_ylim2)
    ax3.set_xlim(*current_xlim)

# Connect the on_zoom function to the zoom event
fig.canvas.mpl_connect('resize_event', on_zoom)

# Save the plot to a file (e.g., PNG)
plt.savefig('heatmap.pdf')

# # Display the saved plot in the notebook
# display(Image(filename='sample_plot.png'))
# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#@title Cluster matches bar plot
#combined bar plot
%matplotlib widget

from google.colab import output
output.enable_custom_widget_manager()
import matplotlib.pyplot as plt
import math
from matplotlib.widgets import Slider
from ipywidgets import *

fig,ax=plt.subplots(figsize=(10,6))
fig.patch.set_facecolor('white')

# Simple mouse click function to store coordinates
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata

    # assign global variable to access outside of function
    global coords
    coords.append((ix, iy))

    # Disconnect after x clicks
    if len(coords) == 100:
        fig.canvas.mpl_disconnect(cid)
        plt.close(1)
    return

def on_move(event):
    if event.inaxes:
        print(f'data coords {event.xdata} {event.ydata},',
              f'pixel coords {event.x} {event.y}')


x = np.arange(qid.max()+1)
y1 = matchhit_array
y2 = matchpair_array

N=100

def bar(pos):
    pos = int(pos)
    ax.clear()
    if pos+N > len(x):
        n=len(x)-pos
    else:
        n=N
    X=x[pos:pos+n]
    Y=y1[pos:pos+n]
    Y2=y2[pos:pos+n]
    ax.bar(X,Y2,width=1,align='edge',ecolor='black',color='lightgrey')
    ax.bar(X,Y,width=0.5,align='edge',ecolor='black')

    # ax.axhline(10,color='r', linestyle='--')

#     ax.set_yscale('log')

barpos = plt.axes([0.18, 0.05, 0.55, 0.03], facecolor="skyblue")
slider = Slider(barpos, 'Gene pos', 50, len(x)-N, valinit=0)
slider.on_changed(bar)

bar(0)

coords = []

# Call click func
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

In [ ]:
#@title Select proteins(s) of interest, extract all cluster matches containing the protein-encoded gene(s).

#@markdown Input: Query Protein ID (single integer or range, e.g., '1' or '1-10')

# Query protein ID parameter
query_protein_id_input = '1' #@param {type:"string"}

# Function to parse the input string to get lower and upper bounds
def parse_input(input_str):
    parts = input_str.split('-')
    if len(parts) == 1:
        # Single integer
        return [int(parts[0])]
    elif len(parts) == 2:
        # Range
        return list(range(int(parts[0]), int(parts[1]) + 1))
    else:
        raise ValueError("Invalid input format. Please enter either a single integer or a range.")

# Print all numbers within the range
query_protein_id = parse_input(query_protein_id_input)

# Filter clusterid to include only clusters with all query_protein_ids
clusterid = matchhit.loc[matchhit['qid'].isin(query_protein_id), 'cluid'].unique().tolist()

# Check if all query_protein_ids are present in each cluster
filtered_clusterid = [cluster for cluster in clusterid if all(matchhit[matchhit['cluid'] == cluster]['qid'].isin(query_protein_id))]

appended_data = pd.DataFrame()
for i in clusterid:
    appended_data = pd.concat([appended_data, matchhit[matchhit['cluid'] == i]])

appended_data = appended_data[appended_data['qseqid'].map(appended_data['qseqid'].value_counts()) > 1]
# query_protein_id = 1 #@param {type:"integer"}
# clusterid = matchhit['cluid'][matchhit['qid'] ==query_protein_id].values.tolist()
# appended_data = pd.DataFrame()
# for i in clusterid:
#     appended_data = pd.concat([appended_data, matchhit[matchhit['cluid'] == i]])


In [ ]:
#@title Convert to R dataframe
%reload_ext rpy2.ipython
from rpy2.robjects import pandas2ri
# clusterid = matchhit['cluid'][matchhit['qid'] ==query_protein_id[0]].values.tolist()
# appended_data = pd.DataFrame()
# for i in clusterid:
#     appended_data = appended_data.append(matchhit[matchhit['cluid'] == i],ignore_index=True)

# condition = np.sign(appended_data['qstart']-appended_data['qend']) != np.sign(appended_data['tstart']-appended_data['tend'])
# appended_data['tstart'] = np.where(condition, -appended_data['tstart'],appended_data['tstart'])
# appended_data['tend'] = np.where(condition, -appended_data['tend'],appended_data['tend'])

predefined_qseqid = query_protein_id[0]

def invert_sign(group):
    matching_rows = group[group['qseqid'] == predefined_qseqid]

    if not matching_rows.empty:
        predefined_row = matching_rows.iloc[0]
        q_direction = np.sign(predefined_row['qstart'] - predefined_row['qend'])
        t_direction = np.sign(predefined_row['tstart'] - predefined_row['tend'])

        if q_direction != t_direction:
            group['tstart'], group['tend'] = -group['tstart'].values, -group['tend'].values

    return group

appended_data_g = appended_data.groupby('cluid', group_keys=False).apply(invert_sign)

center = str(query_protein_id[0])
appended_data_g = appended_data_g[appended_data_g['tname']!= appended_data_g['qname']]
gggene_df = appended_data_g.groupby(by="qid", as_index = False).first()[['qname','qid','qstart','qend']]
gggene_df = gggene_df.append(appended_data_g[['tname','qid','tstart','tend']].rename(columns={"tname": "qname", "tstart": "qstart","tend": "qend"})).reset_index()
pandas2ri.activate()
r_dataframe = pandas2ri.py2rpy(gggene_df)


In [ ]:
#@title Visualize genome context
%%R -i r_dataframe,center
install.packages("gggenes",quiet = TRUE)
library(gggenes)
library(ggplot2)
library(RColorBrewer)
library(IRdisplay)

nb.cols <- length(unique(r_dataframe$qid))
mycolors <- colorRampPalette(brewer.pal(8, "Set3"))(nb.cols)

dummies <- make_alignment_dummies(
  r_dataframe,
  aes(xmin = qstart, xmax = qend, y = qname, id = qid),
  on = center
)

ggplot(r_dataframe, aes(xmin = qstart, xmax = qend, y = qname, fill = factor(qid))) +
  geom_gene_arrow() +
  geom_blank(data = dummies) +
  facet_wrap(~ qname, scales = "free", ncol = 1) +
  scale_fill_manual(values = mycolors) +
  theme_genes()+
  labs(fill = "Gene", y = "Genome")

width <- 1.5 * length(unique(r_dataframe$qid))
height <- 0.5 * length(unique(r_dataframe$qname))

# Save the ggplot to a PDF file (temporarily)
ggsave("spacedust_plot.png", width = width, height = height)

In [ ]:
#@title View genome context and save image
from IPython.display import Image
from google.colab import files


image_path = "/content/spacedust_plot.png"

# Trigger download of the image
files.download(image_path)

# Display the image
Image(filename=image_path, width=500)



# **Instructions**
**Quick start**
1. Set parameters. Press "Runtime" -> "Run all".
2. Wait for file input box to appear below this cell to upload 1) all your query genome files and 2) your target genome files
3. The currently running step is indicated by a circle with a stop sign next to it.

**Result file contents**

1. Tab-separated text file (`.tsv`) of all reported cluster matches.
2. (If applicable) Protein sequences (`.faa`) predicted by prodigal.

<!--At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Additionally, if the `save_to_google_drive` option was selected, the `jobname.result.zip` will be uploaded to your Google Drive. -->

**Troubleshooting**
<!--* If you wish to use ProtNLM for function annotation, check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".-->
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check if your input genomes are unzipped.

**Limitations**
* Computing resources: Due to resource limitations, homology search is currently only available with MMseqs2 instead of Foldseek, which might affect sensitivity of detecting remote homology.

**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/soedinglab/spacedust/issues